# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [112]:
problem = '''You are the person in charge of packing in a large company. Your job is to skillfully pack items of various weights in a box with predetermined capacity. The objective is to use as few boxes as possible. There is a set of items and a set of boxes with an upper bound on the amount of boxes. Each of the items has a known weight. Each item is packed into one box and items are packed into the chosen boxes. Please formulate a mathematical optimization model for this problem. '''

## 2. Generate the mathematical model

In [113]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [114]:
#Show the resopnse in a formatted way
Markdown(response.text)

### Mathematical Optimization Model for Packing Items into Boxes:

**Sets:**

*  I: Set of items  
*  B: Set of boxes

**Parameters:**

*  w_i: Weight of item i  ∈ I
*  C: Capacity of each box
*  N: Upper bound on the number of boxes 

**Decision Variables:**

*  x_ij: Binary variable, equals 1 if item i ∈ I is packed into box j ∈ B, and 0 otherwise. 
*  y_j: Binary variable, equals 1 if box j ∈ B is used, and 0 otherwise.

**Objective Function:**
Minimize the number of boxes used:

```
Minimize Σ_(j ∈ B) y_j 
```

**Constraints:**

1. **Capacity Constraint:** The total weight of items packed into a box cannot exceed the box's capacity.
   ``` 
   Σ_(i ∈ I) w_i * x_ij ≤ C * y_j,  ∀ j ∈ B 
   ```

2. **Item Assignment Constraint:** Each item must be packed into exactly one box.
   ```
   Σ_(j ∈ B) x_ij = 1,  ∀ i ∈ I
   ```

3. **Box Usage Constraint:**  A box can only be used if at least one item is packed into it. 
   ```
   y_j ≤ Σ_(i ∈ I) x_ij,   ∀ j ∈ B
   ```

4. **Box Limit Constraint:** The number of boxes used cannot exceed the upper bound. 
   ```
    Σ_(j ∈ B) y_j ≤ N
   ```

This model represents a classic binary integer programming problem, which can be solved using optimization software packages to find the optimal packing strategy that minimizes the number of boxes used while respecting the problem's constraints. 


## 3. Generate the pyomo code

In [115]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [116]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with your actual data
items = ['A', 'B', 'C', 'D', 'E']
weights = {'A': 10, 'B': 5, 'C': 8, 'D': 12, 'E': 7} 
box_capacity = 20
max_boxes = 5

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items)
model.B = pyo.Set(initialize=range(1, max_boxes + 1)) 

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=box_capacity)
model.N = pyo.Param(initialize=max_boxes)

# Variables
model.x = pyo.Var(model.I, model.B, within=pyo.Binary)
model.y = pyo.Var(model.B, within=pyo.Binary)

# Objective function
def objective_rule(model):
    return sum(model.y[j] for j in model.B)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def capacity_rule(model, j):
    return sum(model.w[i] * model.x[i,j] for i in model.I) <= model.C * model.y[j]
model.capacity_constraint = pyo.Constraint(model.B, rule=capacity_rule)

def item_assignment_rule(model, i):
    return sum(model.x[i,j] for j in model.B) == 1
model.item_assignment_constraint = pyo.Constraint(model.I, rule=item_assignment_rule)

def box_usage_rule(model, j):
    return model.y[j] <= sum(model.x[i,j] for i in model.I) 
model.box_usage_constraint = pyo.Constraint(model.B, rule=box_usage_rule)

def box_limit_rule(model):
    return sum(model.y[j] for j in model.B) <= model.N
model.box_limit_constraint = pyo.Constraint(rule=box_limit_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver 
solver.solve(model) 

# Print the results
print("Optimal Solution:")
for j in model.B:
    if pyo.value(model.y[j]) == 1:
        print(f"Box {j}:", end=" ")
        for i in model.I:
            if pyo.value(model.x[i,j]) == 1:
                print(i, end=" ")
        print()
print(f"Minimum Boxes Used: {pyo.value(model.obj)}") 
```

**Explanation:**

1. **Sample Data:** The code uses sample data for item names (`items`), their corresponding weights (`weights`), box capacity (`box_capacity`), and maximum number of boxes (`max_boxes`). You should replace this with your actual data.

2. **Pyomo Model:** The code defines a Pyomo ConcreteModel to represent the optimization problem.

3. **Sets and Parameters:** It defines sets for items (I) and boxes (B) and parameters for weights (w), box capacity (C), and the maximum number of boxes (N).

4. **Decision Variables:** `model.x` is a binary variable indicating whether an item is packed into a box, and `model.y` is a binary variable indicating whether a box is used.

5. **Objective Function:** `objective_rule` defines the objective to minimize the number of boxes used.

6. **Constraints:** The code defines four constraints: 
    - `capacity_rule`: Ensures that the total weight in a box does not exceed its capacity.
    - `item_assignment_rule`: Ensures that each item is packed into exactly one box.
    - `box_usage_rule`: Ensures that a box is used only if at least one item is packed into it.
    - `box_limit_rule`: Limits the number of boxes used to the specified upper bound.

7. **Solving and Printing Results:** The code uses the GLPK solver to solve the problem and then prints the optimal solution, showing which items are packed into which boxes and the minimum number of boxes used. 

This code provides a basic structure for solving the box-packing problem using Pyomo. You can modify and extend it further based on your specific needs. 


## 4. Input problem data and try running the generated code

In [118]:
import pyomo.environ as pyo

# # Sample Data - Replace with your actual data
# items = ['A', 'B', 'C', 'D', 'E']
# weights = {'A': 10, 'B': 5, 'C': 8, 'D': 12, 'E': 7} 
# box_capacity = 20
# max_boxes = 5

# Sample Data - Replace with your actual data
items =  range(1,25)
weights = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item

box_capacity = 9
max_boxes = 15

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items)
model.B = pyo.Set(initialize=range(1, max_boxes + 1)) 

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=box_capacity)
model.N = pyo.Param(initialize=max_boxes)

# Variables
model.x = pyo.Var(model.I, model.B, within=pyo.Binary)
model.y = pyo.Var(model.B, within=pyo.Binary)

# Objective function
def objective_rule(model):
    return sum(model.y[j] for j in model.B)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def capacity_rule(model, j):
    return sum(model.w[i] * model.x[i,j] for i in model.I) <= model.C * model.y[j]
model.capacity_constraint = pyo.Constraint(model.B, rule=capacity_rule)

def item_assignment_rule(model, i):
    return sum(model.x[i,j] for j in model.B) == 1
model.item_assignment_constraint = pyo.Constraint(model.I, rule=item_assignment_rule)

def box_usage_rule(model, j):
    return model.y[j] <= sum(model.x[i,j] for i in model.I) 
model.box_usage_constraint = pyo.Constraint(model.B, rule=box_usage_rule)

def box_limit_rule(model):
    return sum(model.y[j] for j in model.B) <= model.N
model.box_limit_constraint = pyo.Constraint(rule=box_limit_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver 
solver.solve(model) 

# Print the results
print("Optimal Solution:")
for j in model.B:
    if pyo.value(model.y[j]) == 1:
        print(f"Box {j}:", end=" ")
        for i in model.I:
            if pyo.value(model.x[i,j]) == 1:
                print(i, end=" ")
        print()
print(f"Minimum Boxes Used: {pyo.value(model.obj)}") 

Optimal Solution:
Box 1: 3 4 9 
Box 2: 18 
Box 3: 2 22 
Box 4: 6 19 
Box 5: 10 14 
Box 6: 24 
Box 7: 12 15 
Box 9: 1 21 
Box 10: 11 17 
Box 11: 5 20 
Box 12: 23 
Box 13: 8 13 
Box 14: 7 16 
Minimum Boxes Used: 13.0


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [119]:
print(response.text)

### Mathematical Optimization Model for Packing Items into Boxes:

**Sets:**

*  I: Set of items  
*  B: Set of boxes

**Parameters:**

*  w_i: Weight of item i  ∈ I
*  C: Capacity of each box
*  N: Upper bound on the number of boxes 

**Decision Variables:**

*  x_ij: Binary variable, equals 1 if item i ∈ I is packed into box j ∈ B, and 0 otherwise. 
*  y_j: Binary variable, equals 1 if box j ∈ B is used, and 0 otherwise.

**Objective Function:**
Minimize the number of boxes used:

```
Minimize Σ_(j ∈ B) y_j 
```

**Constraints:**

1. **Capacity Constraint:** The total weight of items packed into a box cannot exceed the box's capacity.
   ``` 
   Σ_(i ∈ I) w_i * x_ij ≤ C * y_j,  ∀ j ∈ B 
   ```

2. **Item Assignment Constraint:** Each item must be packed into exactly one box.
   ```
   Σ_(j ∈ B) x_ij = 1,  ∀ i ∈ I
   ```

3. **Box Usage Constraint:**  A box can only be used if at least one item is packed into it. 
   ```
   y_j ≤ Σ_(i ∈ I) x_ij,   ∀ j ∈ B
   ```

4. **Box Limit Constr

In [120]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with your actual data
items = ['A', 'B', 'C', 'D', 'E']
weights = {'A': 10, 'B': 5, 'C': 8, 'D': 12, 'E': 7} 
box_capacity = 20
max_boxes = 5

# Model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=items)
model.B = pyo.Set(initialize=range(1, max_boxes + 1)) 

# Parameters
model.w = pyo.Param(model.I, initialize=weights)
model.C = pyo.Param(initialize=box_capacity)
model.N = pyo.Param(initialize=max_boxes)

# Variables
model.x = pyo.Var(model.I, model.B, within=pyo.Binary)
model.y = pyo.Var(model.B, within=pyo.Binary)

# Objective function
def objective_rule(model):
    return sum(model.y[j] for j in model.B)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
def capacity_rule(model, j):
    return sum(model.w[i] * model.x[i,j] for i in model.I) <= model.C * model.y[j]
model.capacity_constraint = pyo.Constraint(model.B, rule=capacity_rule)

def item_assignment_rule(model, i):